En esta primer parte del proyecto nos abocamos a la obtención y depuración de los datos que usaremos para que nuestro chatbot responda. Hemos desarrollado esta parte del proyecto en etapas que describimos a continuación:

**ETAPA 0**: Instalación y Carga básica.

**ETAPA 1**: Transcribimos (convertimos) en texto los videos de las clases que están visibles por medio de YouTube. Para ello guardamos datos de las url de cada video. Además, recopilamos información sobre los participantes del curso y algunos datos del CV del profesor.

**ETAPA 2**: Limpiamos cada uno de los archivos de texto generados a partir de los videos (eliminamos corchetes y símbolos).

**ETAPA 3**: Juntamos todos los archivos de textos (los generados a partir de los videos, los de los datos de participantes del curso, los datos del profesor, los datos de las url de los videos) en un único texto.

**ETAPA 4**: Realizamos una segmemtación (spliteo) del archivo completo con todos los textos juntos (definiendo tramos de 200 caracteres y un solapamineto de 20). Calculamos una aproximación del costo al usar el servicio de OpenIA. Generamos un índice en la nube por medio de Pinecone para guardar los vectores que obtendremos con el embedding. Realizamos el embedding y guardamos los vectores en Pinecone.



**ETAPA 0**

In [ ]:
!pip install langchain -q
!pip install pytube -q
!pip install youtube-transcript-api
!pip install google-colab
from langchain.document_loaders import YoutubeLoader
from pytube import YouTube
from google.colab import files # se utiliza para bajar los archivos txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00


**ETAPA 1**

**1.1** Montamos el drive para acceder al archivo con los datos de url de los videos.

In [ ]:
# Montar Google Drive en Colab
from google.colab import drive
drive.mount('/content/drive')

# Ruta al archivo de texto en Google Drive
file_path = '/content/drive/MyDrive/chatlef/Videos.txt'

**1.2** Preparamos el acceso a los videos de YouTube en formato corto (que es como trabaja YoutubeLoader de langchain).

In [ ]:
# Expresión regular para encontrar URLs en el formato corto (hay que enviarle el formato corto a YoutubeLoader.from_youtube_url)
import re #  Libreria que  utiliza funciones de espresiones regulares para trabajar el tema de las cadenas
urls_youtu_be = []
# Expresión regular para encontrar URLs en el formato largo
regex = r"https://www\.youtube\.com/watch\?v=([\w-]+)"
# Leer las líneas desde el archivo de video
with open(file_path, 'r') as file:
    for line in file:
        # Buscar coincidencias con la expresión regular
        matches = re.findall(regex, line)
        # Convertir las URLs en el formato corto
        urls_short_format = [f"https://youtu.be/{match}" for match in matches]
        # Agregar las URLs en el formato corto a la lista
        urls_youtu_be.extend(urls_short_format)
# Mostrar las URLs en el formato corto
#for url in urls_youtu_be:
 #   print(url)


**1.3** Transcribimos cada video de cada clase del curso, generamos un archivo con la transcripción y los datos de YouTube para cada clase, los guardamos con el nombre de su respectiva url y descargamos los archivos.

In [ ]:
transcripciones = []
for  url in urls_youtu_be:
    # print (url)
     loader=YoutubeLoader.from_youtube_url(url,add_video_info=True,language=["es"])
     transcripcion=loader.load()
     transcripciones.append(transcripcion)
     nombre=transcripcion[0].metadata['source']+".txt"
     # grabar con el nombre de la url
     with open(nombre, 'w') as file:
         file.write(transcripcion[0].page_content)
     # Descargar el archivo .txt
     files.download(nombre)

# Mostrar las transcripciones
for i, transcripcion in enumerate(transcripciones):
    print(f'Transcripción del video {i + 1}:\n{transcripcion}\n')

**ETAPA 2**

Limpiamos cada uno de los archivos de texto generados a partir de los videos

In [ ]:
import os
import re

# Definir una función para eliminar palabras entre corchetes de un texto
def eliminar_palabras_corchetes(texto):
    # Expresión regular para encontrar palabras entre corchetes
    patron = r'\[(.*?)\]'
    # Reemplazar todas las ocurrencias de palabras entre corchetes con una cadena vacía
    texto_sin_corchetes = re.sub(patron, '', texto)
    # Devolver el texto sin palabras entre corchetes
    return texto_sin_corchetes

# Directorio donde se encuentran los archivos de texto
directorio_origen = '/content/drive/MyDrive/chatlef/Txt/'
directorio_destino = '/content/drive/MyDrive/chatlef/Txt/Modificado/'

# Crear el directorio de destino si no existe
if not os.path.exists(directorio_destino):
    os.makedirs(directorio_destino)

# Obtener la lista de archivos en el directorio
archivos_txt = [archivo for archivo in os.listdir(directorio_origen) if archivo.endswith('.txt')]

# Procesar cada archivo de texto
for archivo in archivos_txt:
    # Ruta completa del archivo de origen
    ruta_archivo_origen = os.path.join(directorio_origen, archivo)

    # Leer el contenido del archivo de texto
    with open(ruta_archivo_origen, 'r') as file:
        texto_completo = file.read()

    # Eliminar todas las palabras entre corchetes del texto completo
    texto_sin_corchetes = eliminar_palabras_corchetes(texto_completo)

    # Ruta del archivo de destino
    ruta_archivo_destino = os.path.join(directorio_destino, archivo)

    # Escribir el texto modificado en el archivo de destino
    with open(ruta_archivo_destino, 'w') as file:
        file.write(texto_sin_corchetes)

    print("Palabras entre corchetes eliminadas en el archivo", archivo, "y guardado en", ruta_archivo_destino)


**ETAPA 3**

**3.1** Juntamos todos los archivos de textos que usaremos como datos para ChatLEF.

In [ ]:
from google.colab import drive
import os
from google.colab import files

# Montar Google Drive en el entorno de Colab
drive.mount('/content/drive')

# Ruta de la carpeta que contiene los archivos .txt en Google Drive
carpeta = '/content/drive/My Drive/Archivos/NLP/Modificado'  # RUTA: OJO, PONER LA RUTA CORRECTA SEGUN CADA UNO

# Generar la lista de nombres de archivo en el orden deseado
orden_archivos = ['perfilProfe.txt', 'participantes.txt'] + [f'V{i}.txt' for i in range(1, 67)]

# Lista para almacenar el contenido de todos los archivos
contenido_total = []

# Iterar sobre los archivos en el orden específico
for archivo in orden_archivos:
    ruta_archivo = os.path.join(carpeta, archivo)
    with open(ruta_archivo, 'r') as f:
        contenido_archivo = f.readlines()  # Leer el contenido del archivo
        contenido_total.extend(contenido_archivo)  # Agregar el contenido a la lista total

# Guardar el contenido combinado en un nuevo archivo
ruta_salida = '/content/drive/My Drive/Archivos/NLP/Modificado/contenido_combinado.txt'  # guarda 'contenido_combinado.txt' en el drive
with open(ruta_salida, 'w') as f:
    f.writelines(contenido_total)

# Descargar el archivo combinado
files.download(ruta_salida)


**ETAPA 4**

**4.1** Montamos desde el drive las api keys

In [ ]:
!pip install openai -q
!pip install langchain -q
!pip install tiktoken -q
!pip install pinecone-client -q
#!pip install pinecone-client --upgrade!
!pip install python-dotenv -q
!pip install langchain_pinecone ## Subir a pinecone los vectores


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 4.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # Carga OPENAI_API_KEY
from dotenv import load_dotenv, find_dotenv
load_dotenv('/content/drive/MyDrive/Archivos/NLP/.env')
#os.environ.get("OPENAI_API_KEY")


True

**4.2** Preparamos el archivo con todos los datos para poder segmentarlos usando LangChain

In [ ]:
# Leer el txt contenido_combinado.tx para  extraer el txt en el formato que reconoce
from langchain_community.document_loaders import TextLoader
#loader = TextLoader("/content/drive/MyDrive/chatlef/Txt/Modificado/FINAL/contenido_combinado.txt")
#loader = TextLoader("/content/drive/MyDrive/chatlef/Videos.txt")
loader = TextLoader("/content/drive/MyDrive/Archivos/NLP/Modificado/contenido_combinado.txt")

documento = loader.load()

In [ ]:
# Leer los fragmentos del archivo
# No funciona cuando lo extremos como un archivo texto al parecer necesita la estructura de datos documento[0].page_content ect...
#with open("/content/drive/MyDrive/chatlef/Txt/contenido_unido.txt", "r", encoding="utf-8") as file:
#    documento = file.readlines()

In [ ]:
# Mostramos algunos datos
print(documento)
print(len(documento[0].page_content))

[Document(page_content="Profesor: Gustavo Cabrera.\nTítulo: Analista Universitario de Sistemas, Universidad Tecnológica de Córdoba. Argentina.\nCargos Actuales: Director Académico de IFES Educación Superior, Neuquén, Argentina. Director del Centro de Formación y Empleo en Informática de IFES, Neuquén, Argentina. Cofundador de grupo IFES (Instituto de Formación y Educación Superior), Neuquén, Argentina. \nCargos Anteriores: Director General de Instituto ICONO, Neuquén, Argentina. Analista de Sistemas en el Banco Provincia de Neuquén, Neuquén, Argentina. Programador de Sitemas en XCR, Córdoba, Argentina.\nPerfil Profesional: Especialista en desarrollo, implementación e innovación de propuestas educativas en formatos de estudios presencial, blended learning o distancia. Especialista en diseño de estrategias comerciales y de comunicación para venta de productos de oferta académica. Capacidad de liderazgo, gestión y organización de equipos y grupos de trabajo. Capacidad de desarrollo de pro

In [ ]:
len(documento)

1

**4.3** Realizamos la fragmentación

In [ ]:
# Fragmentacion
from langchain.text_splitter import RecursiveCharacterTextSplitter as RC
text_splitter=RC(chunk_size=200,chunk_overlap=20,length_function=len)
fragmentos=text_splitter.split_documents(documento)
print(len(fragmentos))

8939


In [ ]:
fragmentos[8936].page_content

'sus lecciones muy particulares tengan este tipo de situación a problema pero siempre está bueno que lo que lo investiguen porque esas situaciones cuando menos esperamos siempre aparecen y bueno'

In [ ]:
fragmentos[8935].page_content

'bueno hasta aquí sigamos con la clase de hoy estos tres métodos lo dejo para que ustedes lo pueden investigar y probar son en realidad son muy útiles en tanto en cuanto obviamente sus lecciones muy'

**4.4** Calculamos el costo o presupuesto del uso de OpenIA

In [ ]:
# Para sacar el presupuesto
def print_embedding_cost(texts):
    import tiktoken
    # Obtener el codificador para el modelo de embedding
    enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    # Calcular el número total de tokens para todos los textos
    total_tokens = sum([len(enc.encode(text)) for text in texts])
    # Calcular el costo estimado del embedding (suponiendo $0.0001 por cada 1000 tokens)
    embedding_cost = (total_tokens / 1000) * 0.0001
    # Imprimir el costo total y el costo por token
    print(f"Total Tokens: {total_tokens}")
    print(f"Embedding Cost (USD): {embedding_cost:.5f}")

In [ ]:
# Calcular el costo del embedding de los fragmentos
# aca se complico por que no me toma el array de fragmentos, entonces decidi sacarlo del documento directamente el costo seria mayor por el overlap
print_embedding_cost(documento[0].page_content)

Total Tokens: 1596568
Embedding Cost (USD): 0.15966


**4.5** Generamos indice en Pinecone

In [ ]:
# Conectarse con Pinecone
from pinecone import Pinecone, PodSpec
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [ ]:
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'videos-rujfu32.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'videos',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [ ]:
pc.list_indexes().names()

['videos']

In [ ]:
pc.describe_index("indice")

In [ ]:
for index in pc.list_indexes():
    print(index.name)
    print(index.dimension)
    print(index.metric)
    print(index.status)
    print(index.host)
    print(index.spec)

videos
1536
cosine
{'ready': True, 'state': 'Ready'}
videos-rujfu32.svc.gcp-starter.pinecone.io
{'pod': {'environment': 'gcp-starter',
         'pod_type': 'starter',
         'pods': 1,
         'replicas': 1,
         'shards': 1}}


In [ ]:
#Borrar todos los indices
for i in pc.list_indexes():
    #print(i.name)
    pc.delete_index(i.name)

In [ ]:
# Crear Indice usar (spec=PodSpec) cambio la forma de crearlo
indice="videos"
if indice not in pc.list_indexes().names():
    pc.create_index(
      name=indice,
      dimension=1536,
      metric="cosine",
      spec=PodSpec(
        environment="gcp-starter",
        pod_type="p1.x1",
        pods=1
      )
    )

**4.6** Generamos el embedding y guardamos los vectores en Pinecone

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
#embeddings=OpenAIEmbeddings()
model_name = 'text-embedding-ada-002'
OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")
embeddings= OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
## Subir a pinecone los vectores
from langchain_pinecone import Pinecone
index_name = "videos"
docsearch = Pinecone.from_documents(fragmentos, embeddings, index_name=index_name)

In [ ]:
fragmentos[1].page_content

'Cargos Actuales: Director Académico de IFES Educación Superior, Neuquén, Argentina. Director del Centro de Formación y Empleo en Informática de IFES, Neuquén, Argentina. Cofundador de grupo IFES'

In [ ]:
# Agregar vectores a un indice existente En caso que querramos incorporar mas datos a nuestro embedding
from langchain_pinecone import Pinecone
index_name = "videos"
vectorstore = Pinecone(index_name=index_name, embedding=embeddings)
texts = [doc.page_content for doc in fragmentos]
vectorstore.add_texts(texts)

**HEMOS TERMINADO DE CARGAR NUESTROS DATOS VECTORIZADOS**